# 공공데이터 : 한국토지주택공사_공공임대주택 단지정보 조회 서비스

In [ ]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib
import requests
import json
#from xml.etree.ElementTree import parse
#import xmltodict
import urllib.request
import pandas as pd
import numpy as np
import os

In [246]:
#광역시도코드(city), 시군구코드(sigungu)
base_dir = '/Users/yelinkim/Desktop/DACON'
excel_file = '광역시도_시군구_코드목록.xlsx'
excel_dir = os.path.join(base_dir, excel_file)
tmp = pd.read_excel(excel_dir,header = 1)

city =list(map(str,map(int,tmp['광역시도코드'].dropna())))
sigungu =[[0]]*len(city)
for i in range(8):
    sigungu[i]=list(map(str,map(int,np.split(tmp['시군구코드'],np.where(np.isnan(tmp['시군구코드']))[0])[i+1].dropna())))
sigungu[7] = ['110']#세종특별자치시
for i in range(8,len(city)):
    sigungu[i]=list(map(str,map(int,np.split(tmp['시군구코드'],np.where(np.isnan(tmp['시군구코드']))[0])[i].dropna())))

city_sigungu_code= dict(zip(city, sigungu))


In [275]:
API_URL = "https://data.myhome.go.kr:443/rentalHouseList?"
#API_KEY = '+S/d2yQ1IEXOeKRQu8d7zfzbewcAKC3faJRZIwJGGfipQ5KXwLuuuAOJBwoA0yywxolcfHYT4i4HI58nOJQ6YA=='
API_KEY = 'WEEw6JZGcixn05wZX/0WCx8zguFMryaQqUb53E78PyLS9Ib7KXYzOub72b1A/nTkXxO8NhTREOCF/yj36RA6YA=='
#API_KEY = 'gRxJvCQ58CzcZEhsJafq+Lq9jbvcBmr+EzcA/M+d/X5k5/sU9ITUxdpiAdDwzE4psmBm0yD481e+WUFs6JjcHw=='


In [280]:
final_data = pd.DataFrame()
for city in city_sigungu_code:
    for sigungu in city_sigungu_code[city]:
        
        params = dict()
        params['ServiceKey'] = API_KEY
        #광역시도 코드
        params['brtcCode'] = city
        #시군구 코드
        params['signguCode'] = sigungu
        #조회될 목록의 페이지당 데이터 개수 (기본값:10)
        params['numOfRows'] = '1000000000'
        #조회될 페이지의 번호 (기본값:1)
        params['pageNo'] = '1'
        params['type'] = 'json'
        r = requests.get(API_URL, params=params)
        
        json_object = json.loads(r.text)
        body = json_object['hsmpList']
        data = pd.json_normalize(body)
        final_data= pd.concat([final_data,data],axis=0)


In [281]:
final_data

,numOfRows,pageNo,totalCount,hsmpSn,insttNm,brtcCode,brtcNm,signguCode,signguNm,hsmpNm,...,suplyCmnuseAr,houseTyNm,parkngCo,bassRentGtn,bassMtRntchrg,bassCnvrsGtnLmt,competDe,heatMthdDetailNm,buldStleNm,elvtrInstlAtNm
0,1000000000,1,144,31351683,LH서울,11,서울특별시,110,종로구,서울특별시 종로구,...,12.1124,오피스텔,0,600000,338000,0,NaN,NaN,NaN,NaN
1,1000000000,1,144,31351683,LH서울,11,서울특별시,110,종로구,서울특별시 종로구,...,12.1074,오피스텔,0,600000,312000,0,NaN,NaN,NaN,NaN
2,1000000000,1,144,31351683,LH서울,11,서울특별시,110,종로구,서울특별시 종로구,...,13.2757,오피스텔,0,600000,342000,0,NaN,NaN,NaN,NaN
3,1000000000,1,144,31351683,LH서울,11,서울특별시,110,종로구,서울특별시 종로구,...,13.2706,오피스텔,0,600000,368000,0,NaN,NaN,NaN,NaN
4,1000000000,1,144,31351683,LH서울,11,서울특별시,110,종로구,서울특별시 종로구,...,13.5241,오피스텔,0,600000,363000,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,1000000000,1,165,3689,제주특별자치도개발공사,50,제주특별자치도,130,서귀포시,SK원빌라가동,...,6.7000,다세대주택,0,4314100,18000,0,NaN,NaN,NaN,NaN
161,1000000000,1,165,3688,제주특별자치도개발공사,50,제주특별자치도,130,서귀포시,반석주택나동,...,11.3000,연립주택,0,4147000,26250,0,NaN,NaN,NaN,NaN
162,1000000000,1,165,3687,제주특별자치도개발공사,50,제주특별자치도,130,서귀포시,SK원연립가동,...,6.3000,다세대주택,0,5291000,21670,0,NaN,NaN,NaN,NaN
163,1000000000,1,165,3687,제주특별자치도개발공사,50,제주특별자치도,130,서귀포시,SK원연립가동,...,6.9000,다세대주택,0,5791500,23340,0,NaN,NaN,NaN,NaN


In [285]:
final_data.columns

Index(['numOfRows', 'pageNo', 'totalCount', 'hsmpSn', 'insttNm', 'brtcCode',
       'brtcNm', 'signguCode', 'signguNm', 'hsmpNm', 'rnAdres', 'pnu',
       'hshldCo', 'suplyTyNm', 'styleNm', 'suplyPrvuseAr', 'suplyCmnuseAr',
       'houseTyNm', 'parkngCo', 'bassRentGtn', 'bassMtRntchrg',
       'bassCnvrsGtnLmt', 'competDe', 'heatMthdDetailNm', 'buldStleNm',
       'elvtrInstlAtNm'],
      dtype='object')

In [286]:
final_data.columns = np.array(['numOfRows','pageNo','totalCount','hsmpSn',
'기관명','광역시도코드','광역시도명','시군구코드','시군구명','단지명','도로명주소',
'pnu','세대수','공급유형명','형명','공급전용면적','공급공용면적','주택유형명',
'주차수','기본임대보증금','기본월임대료','기본전환보증금',
'competDe', 'buldStleNm', 'elvtrInstlAtNm',
'heatMthdDetailNm'])


In [288]:
final_data = final_data.drop(columns=['numOfRows','pageNo','totalCount','hsmpSn','pnu','competDe', 'buldStleNm', 'elvtrInstlAtNm',
'heatMthdDetailNm'])

In [291]:
final_data.to_csv('external_data.csv',index=False)